In [36]:
# importing
from bs4 import BeautifulSoup, SoupStrainer, NavigableString
import requests

In [ ]:
# creating scrapable objects from 2 landing pages (acts and regulations)

landing_pages = ["https://laws-lois.justice.gc.ca/eng/acts/", "https://laws-lois.justice.gc.ca/eng/regulations/"]

# creating a list of doc titles
doc_titles = []
doc_links = []

for landing_page in landing_pages:
  # creating a soup for each page
  page = requests.get(landing_page)
  soup = BeautifulSoup(page.text, features="html.parser")


  alpha_link = ""

  # entering all links for alphabetical act/regulation pages into list
  for item in soup.find_all("a"):
    try:
      if item["class"] == ["btn", "btn-default"]:
        alpha_page = item["href"]
        alpha_link = f"{landing_page}{alpha_page}"

        # soup alphabetical link
        alpha_page = requests.get(alpha_link)
        alpha_soup = BeautifulSoup(alpha_page.text, features="html.parser")

        # soup for each individual act/regulation from alphabetical link
        for item in alpha_soup.find_all("a"):
          if "class" in item.attrs.keys():
            if item.attrs["class"] == ["TocTitle"]:
              if item.string != None:
                doc_titles.append(str(item.string[2:-2]))
                doc_links.append(f"{landing_page}{item.get('href')}")
    except: 
      continue



Access to Information Act


Accessible Canada Act


Addition of Lands to Reserves and Reserve Creation Act


Administrative Tribunals Support Service of Canada Act


Aeronautics Act


Agreements and Conventions


Agricultural and Rural Development Act (ARDA)


Agricultural Marketing Programs Act


Agricultural Products Marketing Act


Agriculture and Agri-Food Administrative Monetary Penalties Act


Air Canada Public Participation Act


Air Travellers Security Charge Act


Airport Transfer (Miscellaneous Matters) Act


Alberta Natural Resources Act


Alternative Fuels Act


Animal Pedigree Act


Anishinabek Nation Education Agreement Act


Anishinabek Nation Governance Agreement Act


Annulment of Marriages Act (Ontario)


Antarctic Environmental Protection Act


Anti-Personnel Mines Convention Implementation Act


Anti-terrorism Act


Apprentice Loans Act


Appropriation Acts


Arctic Waters Pollution Prevention Act


Asian Infrastructure Investment Bank Agreement Act


Asia-Pacific 

In [ ]:
doc_titles.append("The Constitution Acts 1867 to 1982")
doc_links.append("https://laws-lois.justice.gc.ca/eng/Const/index.html")

4771
4771


In [105]:
with open("data/names.txt", "a") as file, open("data/links.txt", "a") as file1:
  for i in range(3):  # how many acts/regulations are being looped through
    name = doc_titles[i]
    link = doc_links[i]

    scrapable_link = ""
    try:
      scrapable_link = requests.get(f"{link[:-11]}/FullText.html")  # getting rid of /.index.html from the end and replacing with /FullText.html
      scrapable_link.encoding = scrapable_link.apparent_encoding  # utf-8 encoding
      scrapable_soup = BeautifulSoup(scrapable_link.text, "html.parser")
      # if the page can be found, append title to names.txt

      file.write(f"{doc_titles[i]}\n")
      file1.write(f"{doc_links[i]}\n")
    
      # write scraped data into associated file
      with open(f"data/scraped_content/{doc_titles[i].replace(' ', '_')}.txt", "a", encoding="utf-8") as file2:
        for item in scrapable_soup.find_all(["p", "h2", "h3", "h4"]):
          if (item.name == "p"):
            for descendant in item.descendants:
              if isinstance(descendant, NavigableString):
                if str(descendant) != "Marginal note:":
                  file2.write(descendant)
            file2.write("\n")
          
          elif (item.name == "h2"):
            try:
              if "Part" in item["class"]:
                for descendant in item.descendants:
                  if isinstance(descendant, NavigableString):
                    if str(descendant) != "Marginal note:":
                      file2.write(descendant)
                file2.write("\n")
            except:
              continue
          
          elif (item.name ==  "h3" or item.name == "h4"):
            try:
              if "Subheading" in item["class"]:
                for descendant in item.descendants:
                  if isinstance(descendant, NavigableString):
                    if str(descendant) != "Marginal note:":
                      file2.write(descendant)
                file2.write("\n")
            except:
              continue

    except:
      continue